In [1]:
# follow the steps and modified from the github eval_*.py
# Github: https://github.com/titu1994/neural-image-assessment

import numpy as np
import argparse
from tqdm import tqdm

import tensorflow as tf
import os

from keras.models import Model
from keras.layers import Dense, Dropout
from keras.preprocessing.image import load_img, img_to_array

from utils.nasnet import NASNetMobile, preprocess_input
from utils.score_utils import mean_score, std_score

E:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5124965882682021193
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3174131302
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16157427006511274458
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
target_size = (224, 224)  # NASNet requires strict size set to 224x224

In [4]:
import os
from zipfile import ZipFile
import cv2
import numpy as np
import pandas as pd
from dask import bag, threaded
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

In [5]:
def load_img_from_zipped(zipped, filename):
    try:
        exfile = zipped.read(filename)
        arr = np.frombuffer(exfile, np.uint8)
        imz = cv2.imdecode(arr, flags=cv2.IMREAD_UNCHANGED)
        imz = cv2.resize(imz, target_size, interpolation=cv2.INTER_AREA)
    except:
        print(filename, ' is invalid')
        imz = None
        
    return imz

In [6]:
# yield image_id, img
def img_loader():
    archives = ['imgs/train_jpg.zip', 'imgs/test_jpg.zip']
    
    invalid_img_ids = ['4f029e2a00e892aa2cac27d98b52ef8b13d91471f613c8d3c38e3f29d4da0b0c', 
                       '8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58', 
                       '60d310a42e87cdf799afcd89dc1b11ae3fdc3d0233747ec7ef78d82c87002e83', 
                       'b98b291bd04c3d92165ca515e00468fd9756af9a8f1df42505deed1dcfb5d7ae']
    
    for arch in archives:
        print('Reading ', arch)
        zipped = ZipFile(arch)
        filenames = zipped.namelist()[1:]
        
        for file in tqdm(filenames):
            img_id = file.replace('.jpg', '')
            
            if img_id in invalid_img_ids:
                continue
                
            img = load_img_from_zipped(zipped, file)
            if img is None:
                continue
                
            yield img_id, img

In [56]:
for
zipped = ZipFile('imgs/test_jpg.zip')
filenames = zipped.namelist()[1:]
filenames

['data/competition_files/test_jpg/4638cc8e7ee42a312ff354e4479fa64d6346aac6cfc161ae47449f21e6519a6c.jpg',
 'data/competition_files/test_jpg/e7ad8bc1bc562f5d8c7447e4c942cebcb01676de3426a947ef2b53b1896be8f9.jpg',
 'data/competition_files/test_jpg/3e8d73bed377ca25264d3280655eea3d639b9e5cb5cca505163f22226e34212d.jpg',
 'data/competition_files/test_jpg/51368d01802656883caf4d73aa51d2d2c571f70505100143148ae36d7b88baa6.jpg',
 'data/competition_files/test_jpg/32254d27ae5794696232b6c8d92b34eb5d5131eb7f99b122e008982aff226787.jpg',
 'data/competition_files/test_jpg/166c7f0adf518cc4ffc8a63ed0cd1df208d433d71645dcb2eed79ec71b52a775.jpg',
 'data/competition_files/test_jpg/9ff2ffa19982994f04ff2eaf936e31a237148bbad1d532a437c181488204f0c9.jpg',
 'data/competition_files/test_jpg/0d72decfed89beb5179bbaeb14578a09303356786703441e288431707940bc63.jpg',
 'data/competition_files/test_jpg/c59289021f8096251521f508960563eb331dc5b73d909e1639a189cc9cb7c4c0.jpg',
 'data/competition_files/test_jpg/5b76a7cf73891898f720f

In [7]:
DEBUG = False

with tf.device('/GPU:0'):
    base_model = NASNetMobile((224, 224, 3), include_top=False, pooling='avg', weights=None)
    x = Dropout(0.75)(base_model.output)
    x = Dense(10, activation='softmax')(x)
    model = Model(base_model.input, x)
    model.load_weights('weights/nasnet_weights.h5')
    print('Weights loaded')
    
    img_ids = []
    score_list = []

    for img_id, img in img_loader():
        
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        scores = model.predict(x, batch_size=1, verbose=0)[0]
        img_ids.append(img_id)
        score_list.append(scores)
        
        if DEBUG:
            print(img_ids)
            print(score_list)
            plt.imshow(img)
            plt.show()
            break

Weights loaded
Reading  imgs/train_jpg_0.zip


100%|████████████████████████████████████████████████████████████████████████| 278167/278167 [2:47:17<00:00, 27.71it/s]


Reading  imgs/train_jpg_1.zip


100%|████████████████████████████████████████████████████████████████████████| 278166/278166 [2:47:35<00:00, 27.66it/s]


Reading  imgs/train_jpg_2.zip


...

100%|████████████████████████████████████████████████████████████████████████| 278166/278166 [2:47:49<00:00, 27.62it/s]


Reading  imgs/train_jpg_4.zip


100%|████████████████████████████████████████████████████████████████████████| 278166/278166 [2:49:36<00:00, 27.33it/s]


Reading  imgs/test_jpg.zip


100%|████████████████████████████████████████████████████████████████████████| 465829/465829 [4:48:41<00:00, 26.89it/s]


In [59]:
img_ids = [img_id.replace('data/competition_files/test_jpg/', '') for img_id in tqdm(img_ids)]


100%|███████████████████████████████████████████████████████████████████| 1856656/1856656 [00:00<00:00, 1990976.92it/s]

In [74]:
img_ids[-3:]

['7abf6c4aa44b9a0d04414c2e350d528e0a83c300f809bd4299226ba2f43e69d6',
 '1ce2716b4022d40c38d18dfbdfb37bbcdadd2a494dc4ce28339f68ab0c8f7cca',
 'bfc4ee7103ffa563152b94b25a29c28d6967ce8e8990072120902313963fd807']

In [14]:
score_list[:3]

[array([0.02021847, 0.05123571, 0.12512006, 0.2761127 , 0.31151772,
        0.13658957, 0.04995457, 0.02059884, 0.00557685, 0.00307551],
       dtype=float32),
 array([0.04311303, 0.08751361, 0.16680562, 0.28526697, 0.2504091 ,
        0.10246787, 0.0404527 , 0.01804111, 0.00350323, 0.00242668],
       dtype=float32),
 array([0.00881213, 0.02849251, 0.07964744, 0.21630877, 0.34663433,
        0.19870226, 0.07647053, 0.03049389, 0.00986233, 0.00457572],
       dtype=float32)]

In [1]:
import pickle

with open('nima_nasnet_img_ids.pickle', 'wb') as handle:
    pickle.dump(img_ids, handle)

with open('nima_nasnet_scores.pickle', 'wb') as handle:
    pickle.dump(score_list, handle)

NameError: name 'img_ids' is not defined

In [1]:
import pandas as pd
import numpy as np
import pickle

with open('nima_nasnet_img_ids.pickle', 'rb') as handle:
    img_ids = pickle.load(handle)

with open('nima_nasnet_scores.pickle', 'rb') as handle:
    score_list = pickle.load(handle)

In [2]:
img_ids[-1]

'bfc4ee7103ffa563152b94b25a29c28d6967ce8e8990072120902313963fd807'

In [3]:
score_list = np.array(score_list)
score_list.shape

(1856656, 10)

In [4]:
#feature engineering
res = pd.DataFrame()
res.loc[:, 'image'] = img_ids
for i in range(10):
    res.loc[:, 'raw_pred_{}'.format(i)] = score_list[:,i]

In [5]:
raw_feature_cols = ['raw_pred_{}'.format(i) for i in range(10)]
raw_feature_cols

['raw_pred_0',
 'raw_pred_1',
 'raw_pred_2',
 'raw_pred_3',
 'raw_pred_4',
 'raw_pred_5',
 'raw_pred_6',
 'raw_pred_7',
 'raw_pred_8',
 'raw_pred_9']

In [6]:
si = np.arange(1,11,1)
print(si)
print(res[raw_feature_cols].values)
print(res[raw_feature_cols].values*si)

[ 1  2  3  4  5  6  7  8  9 10]
[[ 0.02021847  0.05123571  0.12512006 ...,  0.02059884  0.00557685
   0.00307551]
 [ 0.04311303  0.08751361  0.16680562 ...,  0.01804111  0.00350323
   0.00242668]
 [ 0.00881213  0.02849251  0.07964744 ...,  0.03049389  0.00986233
   0.00457572]
 ..., 
 [ 0.03676113  0.08276659  0.17493859 ...,  0.01253165  0.00220958
   0.00137837]
 [ 0.0537097   0.10780383  0.19656125 ...,  0.0101121   0.00146391
   0.00098676]
 [ 0.01191856  0.0326259   0.08416974 ...,  0.03762877  0.01389297
   0.00701537]]
[[ 0.02021847  0.10247141  0.37536018 ...,  0.16479073  0.05019169
   0.0307551 ]
 [ 0.04311303  0.17502722  0.50041687 ...,  0.14432885  0.03152906
   0.02426683]
 [ 0.00881213  0.05698501  0.23894233 ...,  0.2439511   0.08876098
   0.04575717]
 ..., 
 [ 0.03676113  0.16553317  0.52481577 ...,  0.10025321  0.01988618
   0.01378375]
 [ 0.0537097   0.21560766  0.58968374 ...,  0.08089677  0.01317522
   0.00986763]
 [ 0.01191856  0.0652518   0.25250921 ...,  0.30103

In [7]:
si = np.arange(1,11,1)
#res.loc[:, 'real_mean'] = (res[raw_feature_cols].values*si).sum(axis=1)

si = np.repeat([si], res.shape[0],axis=0)
#res.loc[:, 'real_std'] = np.sum(((si-res[['real_mean']].values)**2)*res[raw_feature_cols].values, axis=1)**.5
'''
res.loc[:, 'max_pos'] = np.argmax(res[raw_feature_cols].values, axis=1)
res.loc[:, 'min_pos'] = np.argmin(res[raw_feature_cols].values, axis=1)
'''
res.loc[:, 'med'] = res[raw_feature_cols].median(axis=1)
res.loc[:, 'std'] = res[raw_feature_cols].std(axis=1)
res.loc[:, 'max'] = res[raw_feature_cols].max(axis=1)
res.loc[:, 'min'] = res[raw_feature_cols].min(axis=1)
res.loc[:, '1_quartile'] = res[raw_feature_cols].quantile(.25, axis=1)
res.loc[:, '3_quartile'] = res[raw_feature_cols].quantile(.75, axis=1)
res.loc[:, '13_quartile_diff'] = res.loc[:, '3_quartile']-res.loc[:, '1_quartile']
res.loc[:, 'max_min_diff'] = res.loc[:, 'max'] - res.loc[:, 'min']
res.loc[:, 'non_max_mean'] = (res[raw_feature_cols].sum(axis=1)-res.loc[:, 'max'])/9
res.loc[:, 'max_non_max_mean_diff'] = res.loc[:, 'max']-res.loc[:, 'non_max_mean']

In [8]:
pd.options.display.max_columns=100
res = res.drop(raw_feature_cols, axis=1)
res.head(3)

,image,med,std,max,min,1_quartile,3_quartile,13_quartile_diff,max_min_diff,non_max_mean,max_non_max_mean_diff
0,856e74b8c46edcf0c0e23444eab019bfda63687bb70a34...,0.050595,0.112342,0.311518,0.003076,0.020314,0.133722,0.113409,0.308442,0.076498,0.235020
1,122d198cf11ab32d2346bff455d6702f1ea519df957cea...,0.065313,0.102145,0.285267,0.002427,0.023644,0.150721,0.127077,0.282840,0.079415,0.205852
2,2809fd6afd6d3cae4dd4ad93a7f905a0db32292f4df4b3...,0.053482,0.115737,0.346634,0.004576,0.014520,0.168939,0.154419,0.342059,0.072596,0.274038


In [9]:
for file in ['data/train.csv', 'data/test.csv']:
    if file == 'data/train.csv':
        print('sort by activation date')
        df = pd.read_csv(file, usecols=['image', 'activation_date'], parse_dates=['activation_date']) \
               .sort_values('activation_date') \
               .reset_index(drop=True) \
               .drop('activation_date', axis=1)
    else:
        df = pd.read_csv(file, usecols=['image'])
        
    print('null percentage:', df.isnull().sum().max() / df.shape[0] * 100.)
    print('image in both df and res\n', df.image.isin(res.image).value_counts())
    df = df.merge(res, on='image', how='left').fillna(-1)
    
    print(df.tail(20))
    df.to_csv(file.replace(".csv", "_nasnet_nima_features_ver2.csv"), index=False)

sort by activation date
null percentage: 7.48877229577
image in both df and res
 True     1390827
False     112597
Name: image, dtype: int64
                                                     image       med  \
1503404  2be1b7fddec6a779e5770a39d3a8322a750e4874d4f6ef...  0.068083   
1503405  682fca314e99afb85b8577a6466da8da0bc94e8764f3ce...  0.057518   
1503406  53e10cf76c59451ed9dbe2d313a0c178a7c0db8a01b96e...  0.053425   
1503407  a49b727bb8d37f7a0b1a0f100dbd6ff69b2c914ef9dab6...  0.100741   
1503408  5bdbe60d03ed200de9b27ff1816879c1abdd25300db0c4...  0.063268   
1503409  8b2aba840ba311537a936630be8d3722649006cfefe468...  0.051062   
1503410  cf6a421ff8ce70f0b0faae386b64d6607ecbc89b40cd74...  0.062044   
1503411  4c60a49ca031a0d6f37ae9aee6167bdb0a23d9dbc67d9a...  0.051695   
1503412                                                 -1 -1.000000   
1503413  7d04440d912c9efa92f61709648afd832c4739f2ccf524...  0.052766   
1503414                                                 -1 -1.00000